Nettoyer les Données

Installation des dépendances, téléchargement du modèle spaCy et exécution du script de nettoyage des données

In [ ]:
# Installer les dépendances
!pip install pandas spacy fr_core_news_sm  # ou en_core_web_sm

# Télécharger le modèle spaCy
!python -m spacy download fr_core_news_sm

# Lancer le script
!python clean_data.py

In [ ]:
!pip install langdetect

In [ ]:
!pip install pandas spacy langdetect tqdm
!python -m spacy download fr_core_news_sm

Installation des dépendances, téléchargement du modèle spaCy et exécution du script de nettoyage des données

In [ ]:
# clean_data.py
import pandas as pd
import re
import spacy
from typing import List, Dict
from unicodedata import normalize
from langdetect import detect
import warnings
from tqdm import tqdm  # Pour une barre de progression

# Configuration initiale
warnings.filterwarnings("ignore", category=UserWarning)
tqdm.pandas()  # Active la barre de progression pour pandas

# Initialiser spaCy avec une limite étendue
try:
    nlp = spacy.load("fr_core_news_sm")
    nlp.max_length = 2000000  # Double la limite de caractères
except OSError:
    print("⚠️ Modèle spaCy non trouvé. Veuillez l'installer avec :")
    print("python -m spacy download fr_core_news_sm")
    exit(1)

def is_chinese(text: str) -> bool:
    """Détecte si le texte est en chinois"""
    try:
        if not isinstance(text, str) or len(text.strip()) < 10:
            return False
        # On vérifie seulement les premiers 1000 caractères pour la détection de langue
        return detect(text[:1000]) == 'zh'
    except:
        return False

def normalize_text(text: str) -> str:
    """
    Nettoie et normalise le texte avec gestion des longs textes
    """
    if not isinstance(text, str) or is_chinese(text):
        return ""

    # Limite la taille du texte pour spaCy (500k caractères max)
    processing_text = text[:500000]

    # Normalisation Unicode
    processing_text = normalize("NFKD", processing_text).encode("ASCII", "ignore").decode("utf-8")

    # Nettoyage de base
    processing_text = re.sub(r"http\S+|@\w+|#\w+", "", processing_text)
    processing_text = re.sub(r"[^a-zA-Z0-9\séèêëàâäîïôöùûüç]", " ", processing_text)

    # Traitement par morceaux si le texte est trop long
    chunk_size = 100000
    text_chunks = [processing_text[i:i+chunk_size]
                  for i in range(0, len(processing_text), chunk_size)]

    final_tokens = []
    for chunk in text_chunks:
        doc = nlp(chunk.lower())
        final_tokens.extend([token.lemma_ for token in doc
                           if not token.is_stop and not token.is_punct])

    return " ".join(final_tokens).strip()

def preprocess_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Applique le prétraitement de manière optimisée"""
    # 1. Nettoyage initial
    df = df.drop_duplicates(subset=["URL"])
    df = df.dropna(subset=["Contenu", "Titre"])
    print(f"📊 Après nettoyage initial : {len(df)} articles")

    # 2. Filtrage des articles en chinois
    print("🔍 Filtrage des articles en chinois...")
    df['is_chinese'] = df['Contenu'].progress_apply(is_chinese)
    chinese_count = df['is_chinese'].sum()
    print(f"🚮 {chinese_count} articles en chinois détectés et supprimés")
    df = df[~df['is_chinese']].copy()

    # 3. Normalisation du texte avec barre de progression
    text_columns = ["Titre", "Auteur", "Description", "Contenu"]
    for col in text_columns:
        if col in df.columns:
            print(f"🔄 Normalisation de la colonne {col}...")
            df[col] = df[col].progress_apply(normalize_text)

    # 4. Formatage des dates
    if "Date" in df.columns:
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce").dt.strftime("%Y-%m-%d")

    # 5. Ajout d'ID
    df["ID"] = range(1, len(df) + 1)

    return df[["ID", "Titre", "Auteur", "Date", "Source", "URL", "Contenu"]]

if __name__ == "__main__":
    try:
        print("📂 Chargement des données...")
        df = pd.read_csv("/content/drive/MyDrive/merged_articles.csv")
        print(f"🔍 Données brutes chargées : {len(df)} articles")

        cleaned_df = preprocess_dataframe(df)

        print("💾 Sauvegarde des données nettoyées...")
        cleaned_df.to_csv("/content/drive/MyDrive/cleaned_articles.csv", index=False)
        print(f"✅ {len(cleaned_df)} articles sauvegardés")
        print("📊 Aperçu final :")
        print(cleaned_df.head(3).to_markdown(tablefmt="grid"))

    except Exception as e:
        print(f"❌ Erreur : {str(e)}")

Construction du Système RAG

Installation des packages essentiels pour traitement de données et indexation vectorielle (FAISS CPU et GPU optionnel)

In [ ]:
!pip install sentence-transformers faiss-cpu numpy pandas tqdm
# Pour GPU NVIDIA (optionnel) :
!pip install faiss-gpu

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/cleaned_articles.csv')
df

In [ ]:
print(df.columns)

Système RAG complet : génération d’embeddings, indexation FAISS, recherche sémantique interactive et sauvegarde dans Google Drive

In [ ]:
# rag_system.py
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from tqdm import tqdm
import os

# Configuration des chemins Google Drive
DRIVE_PATH = "/content/drive/MyDrive/rag_project"
os.makedirs(DRIVE_PATH, exist_ok=True)

# 1. Charger les données nettoyées
INPUT_PATH = "/content/drive/MyDrive/cleaned_articles.csv"
print(f"📂 Chargement des données depuis {INPUT_PATH}...")
df = pd.read_csv(INPUT_PATH)

# Vérification des colonnes disponibles
print("📊 Colonnes disponibles:", df.columns.tolist())

# Utilisation des colonnes existantes
texts = df["Contenu"].fillna("").tolist()
metadata_columns = ["ID", "Titre", "URL"]  # Colonnes obligatoires
metadata = df[metadata_columns].to_dict('records')

# 2. Génération des embeddings
EMBEDDINGS_PATH = f"{DRIVE_PATH}/article_embeddings.npy"
print("🔧 Création des embeddings...")
model = SentenceTransformer('all-MiniLM-L6-v2')

# Génération par batch pour les grands datasets
embeddings = []
batch_size = 32 if len(texts) > 1000 else 64

for i in tqdm(range(0, len(texts), batch_size), desc="Embedding des articles"):
    batch = texts[i:i + batch_size]
    embeddings.append(model.encode(batch, show_progress_bar=False))

embeddings = np.vstack(embeddings)
np.save(EMBEDDINGS_PATH, embeddings)
print(f"💾 Embeddings sauvegardés dans {EMBEDDINGS_PATH}")

# 3. Création de l'index FAISS
INDEX_PATH = f"{DRIVE_PATH}/faiss_index.index"
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)
faiss.write_index(index, INDEX_PATH)
print(f"🏗️ Index FAISS sauvegardé dans {INDEX_PATH}")

# 4. Fonction de recherche améliorée
def search(query: str, top_k: int = 5, min_similarity: float = 0.5):
    """Recherche les articles les plus pertinents"""
    try:
        query_embedding = model.encode([query])
        distances, indices = index.search(query_embedding, top_k)

        results = []
        for idx, score in zip(indices[0], distances[0]):
            if idx >= 0 and score >= min_similarity:
                result = metadata[idx].copy()
                result["score"] = float(score)

                # Ajout d'un extrait du contenu (premières 100 caractères)
                result["extrait"] = texts[idx][:100] + "..." if len(texts[idx]) > 100 else texts[idx]
                results.append(result)

        return sorted(results, key=lambda x: x["score"], reverse=True)
    except Exception as e:
        print(f"❌ Erreur lors de la recherche: {str(e)}")
        return []

# 5. Test du système
print("\n🧪 Phase de test - Tapez 'exit' pour quitter")
while True:
    query = input("\n🔎 Entrez votre requête: ")
    if query.lower() == 'exit':
        break

    results = search(query)

    if not results:
        print("Aucun résultat trouvé. Essayez avec d'autres termes.")
        continue

    print(f"\n📚 Meilleurs résultats pour '{query}':")
    for i, res in enumerate(results, 1):
        print(f"\n{i}. {res['Titre']} (score: {res['score']:.2f})")
        print(f"   📝 Extrait: {res['extrait']}")
        print(f"   🔗 Lien: {res['URL']}")

print("\n✅ Système RAG prêt! Tous les fichiers sont sauvegardés dans Google Drive:")
print(f"- Index FAISS: {INDEX_PATH}")
print(f"- Embeddings: {EMBEDDINGS_PATH}")
print(f"- Données originales: {INPUT_PATH}")

Script de test du système RAG : chargement de l’index FAISS, recherche sémantique et affichage interactif des résultats

In [ ]:
# rag_test.py
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd

# Chemins vers les fichiers sauvegardés
DRIVE_PATH = "/content/drive/MyDrive/rag_project"
INDEX_PATH = f"{DRIVE_PATH}/faiss_index.index"
EMBEDDINGS_PATH = f"{DRIVE_PATH}/article_embeddings.npy"
DATA_PATH = "/content/drive/MyDrive/cleaned_articles.csv"

# Charger les ressources existantes
print("⚙️ Chargement des ressources...")
model = SentenceTransformer('all-MiniLM-L6-v2')
index = faiss.read_index(INDEX_PATH)
df = pd.read_csv(DATA_PATH)
texts = df["Contenu"].fillna("").tolist()
metadata = df[["ID", "Titre", "URL"]].to_dict('records')

def search(query: str, top_k: int = 3):
    """Fonction de recherche optimisée"""
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)

    results = []
    for idx, score in zip(indices[0], distances[0]):
        if idx >= 0:
            result = metadata[idx].copy()
            result["score"] = float(score)
            result["extrait"] = texts[idx][:150] + "..." if len(texts[idx]) > 150 else texts[idx]
            results.append(result)

    return sorted(results, key=lambda x: x["score"], reverse=True)

# Interface de test
print("\n🔍 Testez votre système RAG (tapez 'exit' pour quitter)")
while True:
    query = input("\nEntrez votre requête : ")
    if query.lower() == 'exit':
        break

    results = search(query)

    if not results:
        print("Aucun résultat trouvé.")
        continue

    print(f"\n🔎 {len(results)} résultats pour '{query}':")
    for i, res in enumerate(results, 1):
        print(f"\n{i}. [{res['score']:.2f}] {res['Titre']}")
        print(f"   {res['extrait']}")
        print(f"   {res['URL']}")

print("\n✅ Test terminé")

In [ ]:
!pip install langchain-community openai

In [ ]:
!pip install langchain-community openai langchain-huggingface faiss-cpu

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
import os
print("Fichiers dans rag_project:", os.listdir("/content/drive/MyDrive/rag_project"))

Démarrage asynchrone du serveur Ollama via un thread séparé

In [ ]:
import subprocess
import threading

def run_ollama():
    subprocess.run(["ollama", "serve"], check=True)

# Démarrer dans un thread séparé
threading.Thread(target=run_ollama, daemon=True).start()

In [ ]:
!curl http://localhost:11434
# Doit retourner "Ollama is running"

In [ ]:
!ollama pull mistral

Chatbot interactif pour recherche d’articles avec embeddings SentenceTransformer et index FAISS

In [ ]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# Charger les ressources
DRIVE_PATH = "/content/drive/MyDrive/rag_project"
INDEX_PATH = f"{DRIVE_PATH}/faiss_index.index"
DATA_PATH = "/content/drive/MyDrive/cleaned_articles.csv"

print("⚙️ Chargement des ressources...")
model = SentenceTransformer('all-MiniLM-L6-v2')
index = faiss.read_index(INDEX_PATH)
df = pd.read_csv(DATA_PATH)
texts = df["Contenu"].fillna("").tolist()
metadata = df[["ID", "Titre", "URL"]].to_dict('records')

def search_articles(query: str, top_k: int = 5):
    """Recherche les articles les plus pertinents"""
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    results = []
    for idx, score in zip(indices[0], distances[0]):
        if idx >= 0:
            result = metadata[idx].copy()
            result["score"] = float(score)
            result["extrait"] = texts[idx][:200] + "..." if len(texts[idx]) > 200 else texts[idx]
            results.append(result)
    return sorted(results, key=lambda x: x["score"], reverse=True)

def main():
    print("\n=== CHATBOT D'ARTICLES ===")
    while True:
        question = input("\nPose ta question (ou 'exit'): ").strip()
        if question.lower() == 'exit':
            break
        articles = search_articles(question, top_k=5)
        if not articles:
            print("Aucun article trouvé.")
            continue
        print(f"\n{len(articles)} articles trouvés pour ta question :\n")
        for i, art in enumerate(articles, 1):
            print(f"{i}. {art['Titre']}")
            print(f"   🔗 Lien : {art['URL']}")
            print(f"   📝 Résumé : {art['extrait']}\n")

if __name__ == "__main__":
    main()

In [ ]:
from google.colab import files
files.download('/content/drive/MyDrive/merged_articles.csv')
